# ML Foreclosure Binary Classification - EPOCH1 5% sample dataset
Only "resolved" outcomes - Either paid off or default 

    target = zeroBalCode (set to object dtype) 
    Binary classification sets zeroBalCode to either "0" or "1" for the 7 possible outcomes
    
    "0" = "Loan Success"
        * 01 = Prepaid or Matured
        * 06 = Repurchased
        * 16 = Reperforming Loan Sale

    "1" =  "Loan Failure" - Most likely loss of money (but not necessarily) one of below conditions:
        * 02 = Third Party Sale
        * 03 = Short Sale
        * 09 = Deed-in-Lieu,REO
        * 15 = Note Sale


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#!pip install pycaret
from imblearn.over_sampling import RandomOverSampler
from pycaret.classification import *
#from pycaret.regression import *
pd.set_option('display.max_columns', None)


# Importing the data - don't use *.ML.csv files!!!
If you have *.MLReady.csv then [click here](#MLReady)

In [2]:
df=pd.read_csv("data/FM_Acq2_Perf_FMAC_EPOCH1_loanAge.csv")
df.head()

,Unnamed: 0,id,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origDate,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,loanAge,monMatur,mSA,zeroBalCode,deliqGood,deliqBad,deliqMax,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct
0,0,102718910405,R,"JPMORGAN CHASE BANK, NA",8.500,187000,360,2000-02-01,80.0,90.0,2.0,31.0,P,MI,483,0.0,0.0,712.0,706.0,709.0,70,290.0,19820,1.0,50,20,2.0,8.38,8.25,8.325,0.015758,6.62,0.120,-0.250,0.175,0.014320,-0.030303,0.021021
1,1,103123692845,B,"JPMORGAN CHASE BANK, NA",8.500,243000,360,2000-02-01,80.0,95.0,1.0,49.0,P,CA,926,0.0,0.0,682.0,682.0,682.0,21,339.0,31100,1.0,17,4,1.0,8.38,8.25,8.325,0.015758,6.62,0.120,-0.250,0.175,0.014320,-0.030303,0.021021
2,2,117311391991,B,"JPMORGAN CHASE BANK, NA",8.000,253000,360,2000-02-01,79.0,89.0,2.0,34.0,P,CA,953,0.0,0.0,810.0,784.0,797.0,36,324.0,44700,1.0,33,3,1.0,8.38,8.25,8.325,0.015758,6.62,-0.380,0.250,-0.325,-0.045346,0.030303,-0.039039
3,3,135950664937,R,"JPMORGAN CHASE BANK, NA",8.500,138000,360,2000-02-01,80.0,95.0,1.0,30.0,P,MA,17,0.0,0.0,718.0,718.0,718.0,16,344.0,14460,1.0,14,2,1.0,8.38,8.25,8.325,0.015758,6.62,0.120,-0.250,0.175,0.014320,-0.030303,0.021021
4,4,226095365894,B,"JPMORGAN CHASE BANK, NA",8.875,166000,360,2000-02-01,80.0,95.0,2.0,24.0,P,MA,18,0.0,0.0,782.0,752.0,767.0,24,336.0,14460,1.0,23,1,1.0,8.38,8.25,8.325,0.015758,6.62,0.495,-0.625,0.550,0.059069,-0.075758,0.066066


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278367 entries, 0 to 2278366
Data columns (total 38 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   id                  int64  
 2   origChannel         object 
 3   sellerName          object 
 4   origIntRate         float64
 5   origUPB             int64  
 6   origLoanTerm        int64  
 7   origDate            object 
 8   origLTV             float64
 9   origCLTV            float64
 10  numBorrowers        float64
 11  origDebtIncRatio    float64
 12  loanPurp            object 
 13  propState           object 
 14  zipCode             int64  
 15  pMIperct            float64
 16  mortInsType         float64
 17  bestCreditScore     float64
 18  worstCreditScore    float64
 19  avgCreditScore      float64
 20  loanAge             int64  
 21  monMatur            float64
 22  mSA                 int64  
 23  zeroBalCode         float64
 24  deliqGood           int6

## Remove columns not involved in 'Foreclosure or Not' query

In [4]:
df.drop(['Unnamed: 0','id','fredRate',\
         'bestCreditScore','worstCreditScore','monMatur',\
         'deliqGood','deliqBad','deliqMax',\
        'fmacRateMax','fmacRateMin','fmacRateVolatility',\
        'rateDiffAbove','rateDiffBelow','rateDiffAbovePct','rateDiffBelowPct',\
        ],1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278367 entries, 0 to 2278366
Data columns (total 22 columns):
 #   Column            Dtype  
---  ------            -----  
 0   origChannel       object 
 1   sellerName        object 
 2   origIntRate       float64
 3   origUPB           int64  
 4   origLoanTerm      int64  
 5   origDate          object 
 6   origLTV           float64
 7   origCLTV          float64
 8   numBorrowers      float64
 9   origDebtIncRatio  float64
 10  loanPurp          object 
 11  propState         object 
 12  zipCode           int64  
 13  pMIperct          float64
 14  mortInsType       float64
 15  avgCreditScore    float64
 16  loanAge           int64  
 17  mSA               int64  
 18  zeroBalCode       float64
 19  fmacRateAvg       float64
 20  rateDiffAvg       float64
 21  rateDiffAvgPct    float64
dtypes: float64(12), int64(5), object(5)
memory usage: 382.4+ MB


### Check if any columns have a null value

In [6]:
df.apply(lambda x: x.isnull().sum(), axis=0)

origChannel         0
sellerName          0
origIntRate         0
origUPB             0
origLoanTerm        0
origDate            0
origLTV             0
origCLTV            0
numBorrowers        0
origDebtIncRatio    0
loanPurp            0
propState           0
zipCode             0
pMIperct            0
mortInsType         0
avgCreditScore      0
loanAge             0
mSA                 0
zeroBalCode         0
fmacRateAvg         0
rateDiffAvg         0
rateDiffAvgPct      0
dtype: int64

## Engineer originDate into Date features

In [7]:
df['origDate'] = pd.to_datetime(df['origDate'])

In [8]:
df['origYear'] = df['origDate'].dt.year
df['origMonth'] = df['origDate'].dt.month

In [9]:
df.drop(['origDate'],1,inplace=True)

In [10]:
df[['origYear','origMonth']].head()

,origYear,origMonth
0,2000,2
1,2000,2
2,2000,2
3,2000,2
4,2000,2


# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [11]:
df.zeroBalCode.unique()

array([ 1.,  6.,  9.,  3.,  2., 15., 16.])

In [12]:
df['zeroBalCode'] = df.zeroBalCode.replace([1,6,16],0)
df['zeroBalCode'] = df.zeroBalCode.replace([9,3,2,15],1)

In [13]:
df.zeroBalCode.unique()

array([0., 1.])

In [14]:
#Checking datatypes of individual feature
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,zeroBalCode,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth
0,R,"JPMORGAN CHASE BANK, NA",8.500,187000,360,80.0,90.0,2.0,31.0,P,MI,483,0.0,0.0,709.0,70,19820,0.0,8.325,0.175,0.021021,2000,2
1,B,"JPMORGAN CHASE BANK, NA",8.500,243000,360,80.0,95.0,1.0,49.0,P,CA,926,0.0,0.0,682.0,21,31100,0.0,8.325,0.175,0.021021,2000,2
2,B,"JPMORGAN CHASE BANK, NA",8.000,253000,360,79.0,89.0,2.0,34.0,P,CA,953,0.0,0.0,797.0,36,44700,0.0,8.325,-0.325,-0.039039,2000,2
3,R,"JPMORGAN CHASE BANK, NA",8.500,138000,360,80.0,95.0,1.0,30.0,P,MA,17,0.0,0.0,718.0,16,14460,0.0,8.325,0.175,0.021021,2000,2
4,B,"JPMORGAN CHASE BANK, NA",8.875,166000,360,80.0,95.0,2.0,24.0,P,MA,18,0.0,0.0,767.0,24,14460,0.0,8.325,0.550,0.066066,2000,2


# Review values for dates and slice (if needed) the timeframe you want

In [15]:
df.origYear.unique()

array([2000, 1999, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008],
      dtype=int64)

In [16]:
df['origMonth'].unique()

array([ 2, 12, 10,  1, 11,  9,  4,  3,  5,  6,  8,  7], dtype=int64)

In [17]:
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,zeroBalCode,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth
0,R,"JPMORGAN CHASE BANK, NA",8.500,187000,360,80.0,90.0,2.0,31.0,P,MI,483,0.0,0.0,709.0,70,19820,0.0,8.325,0.175,0.021021,2000,2
1,B,"JPMORGAN CHASE BANK, NA",8.500,243000,360,80.0,95.0,1.0,49.0,P,CA,926,0.0,0.0,682.0,21,31100,0.0,8.325,0.175,0.021021,2000,2
2,B,"JPMORGAN CHASE BANK, NA",8.000,253000,360,79.0,89.0,2.0,34.0,P,CA,953,0.0,0.0,797.0,36,44700,0.0,8.325,-0.325,-0.039039,2000,2
3,R,"JPMORGAN CHASE BANK, NA",8.500,138000,360,80.0,95.0,1.0,30.0,P,MA,17,0.0,0.0,718.0,16,14460,0.0,8.325,0.175,0.021021,2000,2
4,B,"JPMORGAN CHASE BANK, NA",8.875,166000,360,80.0,95.0,2.0,24.0,P,MA,18,0.0,0.0,767.0,24,14460,0.0,8.325,0.550,0.066066,2000,2


# Coerce the column dtype (int in this case)

In [18]:
df = df.astype({'origLTV':'int','origCLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'mortInsType':'int','loanAge':'int',\
               'avgCreditScore':'int','zeroBalCode':'int'})

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278367 entries, 0 to 2278366
Data columns (total 23 columns):
 #   Column            Dtype  
---  ------            -----  
 0   origChannel       object 
 1   sellerName        object 
 2   origIntRate       float64
 3   origUPB           int64  
 4   origLoanTerm      int64  
 5   origLTV           int32  
 6   origCLTV          int32  
 7   numBorrowers      int32  
 8   origDebtIncRatio  int32  
 9   loanPurp          object 
 10  propState         object 
 11  zipCode           int64  
 12  pMIperct          float64
 13  mortInsType       int32  
 14  avgCreditScore    int32  
 15  loanAge           int32  
 16  mSA               int64  
 17  zeroBalCode       int32  
 18  fmacRateAvg       float64
 19  rateDiffAvg       float64
 20  rateDiffAvgPct    float64
 21  origYear          int64  
 22  origMonth         int64  
dtypes: float64(5), int32(8), int64(6), object(4)
memory usage: 330.3+ MB


<a id='MLReady'></a>

# Train Test Split to create holdout dataset outside of PyCaret

In [20]:
good = df.zeroBalCode.value_counts()[0]
bad = df.zeroBalCode.value_counts()[1]
perct_bad = round(bad/(good + bad)*100,2)
print(f'We have {perct_bad}% Foreclosures in our dataset')

We have 15.44% Foreclosures in our dataset


In [21]:
from sklearn.model_selection import train_test_split

training_features, test_features, \
training_target, test_target, = train_test_split(
    df.drop(['zeroBalCode'], axis=1)
    , df['zeroBalCode']
    , test_size = .1
    , random_state=12
)

In [22]:
# Further split the training data into training/test
x_train, x_val, y_train, y_val = train_test_split(
    training_features
    , training_target
    , test_size = .1
    ,random_state=12
)

In [23]:
# For the training data, randomly sample 
# ros = RandomOverSampler(sampling_strategy='minority')
# x_train_res, y_train_res = ros.fit_sample(x_train, y_train)

In [24]:
print('#############################################')
print('Before oversampling: "Closed" crushes "Default" and causes issues:')
print(training_target.value_counts())
# print('')
# print('Before oversampling: "Closed" and "Default" are equal')
# print(y_train_res.value_counts())

#############################################
Before oversampling: "Closed" crushes "Default" and causes issues:
0    1733505
1     317025
Name: zeroBalCode, dtype: int64


# Recombine X_train_res, y_train_res so PyCaret can deal with one df

In [25]:
y_train.shape

(1845477,)

In [26]:
df = x_train.copy()

In [27]:
df['zeroBalCode'] = y_train

In [28]:
df.shape

(1845477, 23)

# Review metrics last time before run (to have record of starting values)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1845477 entries, 947909 to 1263587
Data columns (total 23 columns):
 #   Column            Dtype  
---  ------            -----  
 0   origChannel       object 
 1   sellerName        object 
 2   origIntRate       float64
 3   origUPB           int64  
 4   origLoanTerm      int64  
 5   origLTV           int32  
 6   origCLTV          int32  
 7   numBorrowers      int32  
 8   origDebtIncRatio  int32  
 9   loanPurp          object 
 10  propState         object 
 11  zipCode           int64  
 12  pMIperct          float64
 13  mortInsType       int32  
 14  avgCreditScore    int32  
 15  loanAge           int32  
 16  mSA               int64  
 17  fmacRateAvg       float64
 18  rateDiffAvg       float64
 19  rateDiffAvgPct    float64
 20  origYear          int64  
 21  origMonth         int64  
 22  zeroBalCode       int32  
dtypes: float64(5), int32(8), int64(6), object(4)
memory usage: 281.6+ MB


In [30]:
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,avgCreditScore,loanAge,mSA,fmacRateAvg,rateDiffAvg,rateDiffAvgPct,origYear,origMonth,zeroBalCode
947909,R,"JPMORGAN CHASE BANK, NA",5.625,143000,360,73,73,1,23,R,WA,983,0.0,0,655,48,42660,5.7450,-0.1200,-0.020888,2003,3,0
302531,C,NETBANK FUNDING SERVICES,7.125,133000,360,90,90,1,40,R,MI,484,25.0,1,784,9,19820,6.8175,0.3075,0.045105,2001,9,0
2113882,B,SUNTRUST MORTGAGE INC.,6.875,29000,180,73,73,2,23,C,PA,196,0.0,0,536,64,39740,6.3825,0.4925,0.077164,2007,9,0
1004536,C,SUNTRUST MORTGAGE INC.,5.625,46000,360,55,55,2,26,R,CO,812,0.0,0,774,46,0,5.8125,-0.1875,-0.032258,2003,4,0
840733,B,"WITMER FUNDING, LLC",6.250,300000,360,67,67,2,30,R,CT,69,0.0,0,736,13,14860,5.9160,0.3340,0.056457,2003,1,0


# Pycaret - Setup with categorical definition

### Use complex setup 

In [31]:
model_setup = setup(
    df
    , target = 'zeroBalCode' # PyCaret will list this as "Label"
    , normalize = True
    , transformation = False
    , pca = False 
    , ignore_low_variance = False # Variance is calculated using the ratio of unique values to the number of samples, and the ratio of the most common value to the frequency of the second most common value. 
    , ignore_features = None
    , handle_unknown_categorical = True
    , remove_outliers = True
    , remove_multicollinearity = False
    , polynomial_features = False
    , trigonometry_features = False
    , feature_selection = False
    , feature_interaction = False
    , fix_imbalance = True
    , silent = True
    , profile = False
    # outliers from the training data are removed using PCA linear dimensionality reduction using the Singular Value Decomposition technique.
    , bin_numeric_features = [
        'avgCreditScore'
        ] # Set to True to bin numerics using K Means
    , categorical_features = [
         'origChannel'
        , 'loanPurp'
        , 'origYear'
        , 'origMonth'
        , 'mortInsType'
        , 'origLoanTerm'
        ]
    , high_cardinality_features = [
         'propState'
        , 'sellerName'
        , 'mSA'
        , 'zipCode'
    ]
    , high_cardinality_method = 'clustering'
    , numeric_features = [
        'origIntRate'
        , 'numBorrowers'
        , 'origUPB'
        , 'origLTV'
        , 'origCLTV'
        , 'pMIperct'
        , 'loanAge'
        , 'origDebtIncRatio'
        , 'avgCreditScore'
        , 'rateDiffAvg'
        , 'rateDiffAvgPct'
        , 'fmacRateAvg'
    ]
)

Setup Succesfully Completed!


,Description,Value
0,session_id,1523
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(1845477, 23)"
4,Missing Values,False
5,Numeric Features,14
6,Categorical Features,8
7,Ordinal Features,False
8,High Cardinality Features,True
9,High Cardinality Method,clustering


## Compare Models with no blacklist exclusions

In [ ]:
model_results=compare_models(sort='Recall') #blacklist = ['tr','ransac'],'lar','par','huber','llar','lasso','en','ridge','omp','br','svm'])
model_results

IntProgress(value=0, description='Processing: ', max=176)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:53:07
Status,. . . . . . . . . . . . . . . . . .,Fitting Fold 2 of 10
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression
ETC,. . . . . . . . . . . . . . . . . .,225.74 Minutes Remaining


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


## Create models 

### LGBM Classifier

In [44]:
lgbm = create_model('lightgbm', fold=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8556,0.8063,0.2462,0.5200,0.3342,0.2646,0.2880
1,0.8520,0.8101,0.2330,0.4940,0.3166,0.2454,0.2673
2,0.8640,0.8238,0.2500,0.5893,0.3511,0.2887,0.3220
3,0.8608,0.8141,0.2619,0.5565,0.3561,0.2893,0.3152
4,0.8625,0.8186,0.2751,0.5664,0.3704,0.3034,0.3285
5,0.8611,0.8192,0.2808,0.5543,0.3728,0.3040,0.3263
6,0.8525,0.8229,0.2386,0.4980,0.3227,0.2512,0.2727
7,0.8581,0.8120,0.2386,0.5408,0.3311,0.2649,0.2928
8,0.8575,0.8146,0.2500,0.5344,0.3406,0.2724,0.2972
9,0.8561,0.8030,0.2273,0.5263,0.3175,0.2509,0.2788


## Evaluate Models

In [47]:
evaluate_model(lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## Tune Model

In [49]:
tuned_lgbm = tune_model(lgbm, optimize = 'Recall')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8366,0.7607,0.2973,0.4220,0.3489,0.2587,0.2639
1,0.8246,0.7551,0.2803,0.3728,0.3200,0.2217,0.2246
2,0.8336,0.7653,0.3049,0.4118,0.3504,0.2574,0.2612
3,0.8360,0.7677,0.3207,0.4236,0.3650,0.2729,0.2764
4,0.8335,0.7731,0.2979,0.4089,0.3447,0.2520,0.2562
5,0.8279,0.7539,0.2960,0.3881,0.3358,0.2391,0.2420
6,0.8305,0.7611,0.2860,0.3953,0.3319,0.2376,0.2417
7,0.8332,0.7593,0.3030,0.4103,0.3486,0.2554,0.2593
8,0.8293,0.7518,0.2955,0.3939,0.3377,0.2420,0.2453
9,0.8299,0.7423,0.2481,0.3808,0.3005,0.2085,0.2147


In [51]:
evaluate_model(tuned_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [50]:
bagged_lgbm = ensemble_model(lgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8598,0.8096,0.2595,0.5502,0.3526,0.2852,0.3107
1,0.8492,0.8131,0.2235,0.4739,0.3037,0.2312,0.2518
2,0.8612,0.8265,0.2443,0.5658,0.3413,0.2771,0.3078
3,0.8572,0.8135,0.2524,0.5299,0.3419,0.2730,0.2967
4,0.8578,0.8223,0.2524,0.5341,0.3428,0.2743,0.2987
5,0.8636,0.8192,0.2941,0.5699,0.3880,0.3199,0.3422
6,0.8528,0.8245,0.2481,0.5000,0.3316,0.2593,0.2795
7,0.8564,0.8124,0.2443,0.5265,0.3338,0.2652,0.2899
8,0.8569,0.8135,0.2367,0.5319,0.3277,0.2606,0.2875
9,0.8564,0.8051,0.2273,0.5286,0.3179,0.2516,0.2798


In [52]:
evaluate_model(bagged_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [54]:
boosted_lgbm = ensemble_model(lgbm, method='Boosting')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8528,0.7917,0.2500,0.5000,0.3333,0.2608,0.2807
1,0.8489,0.7975,0.2367,0.4735,0.3157,0.2412,0.2596
2,0.8573,0.8094,0.2822,0.5284,0.3679,0.2957,0.3143
3,0.8522,0.8025,0.2751,0.4949,0.3537,0.2778,0.2931
4,0.8550,0.8093,0.2732,0.5125,0.3564,0.2832,0.3010
5,0.8578,0.8055,0.2865,0.5298,0.3719,0.2997,0.3177
6,0.8472,0.7950,0.2348,0.4627,0.3116,0.2358,0.2530
7,0.8525,0.7969,0.2405,0.4980,0.3244,0.2527,0.2739
8,0.8547,0.8044,0.2538,0.5134,0.3397,0.2684,0.2895
9,0.8461,0.7835,0.2178,0.4528,0.2941,0.2195,0.2381


In [55]:
evaluate_model(boosted_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [56]:
final_lgbm = finalize_model(lgbm)

In [59]:
evaluate_model(final_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

# Predict models

In [60]:
predicted_final_lgbm = predict_model(final_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8803,0.8838,0.2494,0.7977,0.3801,0.3333,0.4033


In [61]:
predicted_lgbm = predict_model(lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8652,0.8121,0.1955,0.636,0.2991,0.247,0.3004


In [62]:
predicted_lgbm_tuned = predict_model(tuned_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8334,0.7494,0.2848,0.4058,0.3347,0.2428,0.2478


In [63]:
predicted_lgbm_bagged = predict_model(bagged_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Bagging Classifier,0.8657,0.8147,0.1791,0.6607,0.2818,0.2338,0.2955


# The golden moment... put 10% sample test data through the models...

# Recombine test_features, test_target at first train/test/split
 This "test" set was split BEFORE Oversampling. PyCaret takes the "train" from this split and splits it internally again. Since we have no control of how PyCaret does this we needed to do a train/test split before we input the training set into PyCaret. Now we will recombine the test_features and test_target into one new df

In [64]:
test_features.shape

(6659, 31)

In [65]:
test_target.shape

(6659,)

In [66]:
test_df = test_features.copy()

In [67]:
test_df['zeroBalCode'] = test_target

In [68]:
test_df.shape

(6659, 32)

In [69]:
totalTestCount = test_df.zeroBalCode.value_counts().sum()
posTestCount = test_df.zeroBalCode.value_counts()[1]
print('#############################################')
print('This "Test" set has not been oversamplied')
print(test_df.zeroBalCode.value_counts())
print(f'Positive samples are {round(posTestCount/totalTestCount*100,2)}%')

#############################################
This "Test" set has not been oversamplied
0    5715
1     944
Name: zeroBalCode, dtype: int64
Positive samples are 14.18%


## Create Confusion Matrix output functions

In [70]:
def calc_confusion(row):
    if ((row['zeroBalCode'] == 0) & (row['Label'] == 0)):
        value = 'TrueNegative'
    elif ((row['zeroBalCode'] == 0) & (row['Label'] == 1)):
        value = 'FalsePositive'
    elif ((row['zeroBalCode'] == 1) & (row['Label'] == 1)):
        value = 'TruePositive'
    elif ((row['zeroBalCode'] == 1) & (row['Label'] == 0)):
        value = 'FalseNegative'
    else:
        value = 'Undefined'
    return value

In [71]:
def cM(df):
    print(f'           ##############################')
    print(f'           #             #              #')
    print(f'         0 #    {df["TrueNegative"]}     #     {df["FalsePositive"]}      #')
    print(f'  True     #             #              #')
    print(f'  Class    ##############################')
    print(f'           #             #              #')
    print(f'         1 #    {df["FalseNegative"]}      #     {df["TruePositive"]}      #')
    print(f'           #             #              #')
    print(f'           ##############################')
    print(f'                  0              1        ')
    print(f'                  Predicted Class           ')


In [72]:
def cMCalc():
    # 0 or 2 = Predict equals truth, 1 = Predict doesn't equal truth
    totalNegative = confusionMatrix['TrueNegative'] + confusionMatrix['FalseNegative']
    totalPositive = confusionMatrix['TruePositive'] + confusionMatrix['FalsePositive']
    total = totalNegative + totalPositive
    trueNegativePct = confusionMatrix['TrueNegative'] / total * 100
    falseNegativePct = confusionMatrix['FalseNegative'] / total * 100
    truePositivePct = confusionMatrix['TruePositive'] / total * 100
    falsePositivePct = confusionMatrix['FalsePositive'] / total * 100

    # Accuracy Calculation
    accuracy = (confusionMatrix['TruePositive'] + confusionMatrix['TrueNegative'])/(totalPositive + totalNegative)
    # Recall Calculation
    recall = (confusionMatrix['TruePositive']/(confusionMatrix['TruePositive'] + confusionMatrix['FalseNegative']))
    # Precision Calculation
    precision = (confusionMatrix['TruePositive']/(confusionMatrix['TruePositive'] + confusionMatrix['FalsePositive']))

    print(f'TrueNegative:  {round(trueNegativePct,2)}%')
    print(f'FalseNegative:  {round(falseNegativePct,2)}%')
    print(f'TruePositive:   {round(truePositivePct,2)}%')
    print(f'FalsePositive: {round(falsePositivePct,2)}%')
    print(f'#####################################################')
    print(f'Accuracy: {round(accuracy,2)*100}%   Recall: {round(recall,2)*100}%   Precision: {round(precision,2)*100}%')
    print(f'#####################################################')

# Evaluate Models

In [74]:
predictions = predict_model(final_lgbm, data=test_df)

In [75]:
pd.set_option('max_columns',None)
predictions.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,loanAge,mSA,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth,zeroBalCode,Label,Score
0,C,NETBANK FUNDING SERVICES,6.625,175000,360,50,50,2,42,R,MA,23,0.0,0,745,720,732,5,14460,6.57,6.34,6.485,0.036278,0.055,-0.285,0.140,0.008371,-0.044953,0.021588,2002,7,0,0,0.0022
1,R,OTHER,4.875,158000,180,45,45,2,8,R,OH,437,0.0,0,805,754,779,138,0,5.70,5.31,5.484,0.073446,-0.825,0.435,-0.609,-0.144737,0.081921,-0.111050,2003,5,0,0,0.0122
2,C,"JPMORGAN CHASE BANK, NA",5.250,102000,360,91,91,1,26,R,OK,741,25.0,1,753,753,753,94,46140,6.34,6.14,6.264,0.032573,-1.090,0.890,-1.014,-0.171924,0.144951,-0.161877,2003,8,0,0,0.0909
3,B,"PNC BANK, N.A.",6.000,187000,360,49,49,2,62,C,CA,919,0.0,0,775,775,775,44,41740,5.97,5.85,5.916,0.020513,0.030,-0.150,0.084,0.005025,-0.025641,0.014199,2003,1,0,0,0.0255
4,C,"CITIMORTGAGE, INC.",6.625,135000,360,40,40,2,31,P,CA,935,0.0,0,788,782,785,32,31100,6.78,6.62,6.682,0.024169,-0.155,-0.005,-0.057,-0.022861,-0.000755,-0.008530,2006,6,0,0,0.0085


In [76]:
results = predictions[['zeroBalCode','Label','Score']]

In [77]:
predictions.Label.value_counts()

0    6656
1       3
Name: Label, dtype: int64

In [78]:
results['Confusion'] = results.apply(calc_confusion, axis=1)

In [79]:
confusionMatrix = results.Confusion.value_counts().to_dict()

In [80]:
cM(confusionMatrix)

           ##############################
           #             #              #
         0 #    5713     #     2      #
  True     #             #              #
  Class    ##############################
           #             #              #
         1 #    943      #     1      #
           #             #              #
           ##############################
                  0              1        
                  Predicted Class           


In [81]:
cMCalc()

TrueNegative:  85.79%
FalseNegative:  14.16%
TruePositive:   0.02%
FalsePositive: 0.03%
#####################################################
Accuracy: 86.0%   Recall: 0.0%   Precision: 33.0%
#####################################################


In [82]:
#save_model(final_nb, 'np.final5pct_092020')